In [1]:
import pandas as pd
from db_queries import get_location_metadata

import warnings
warnings.simplefilter('ignore')


## get pct change in predicted deaths

In [6]:
loc_df = get_location_metadata(location_set_id=111, location_set_version_id=664)
us_locs = loc_df.loc[loc_df['path_to_top_parent'].str.startswith('102,'), 'location_id'].to_list()
wa_locs = loc_df.loc[loc_df['path_to_top_parent'].str.startswith('102,570'), 'location_id'].to_list()

df = pd.read_csv('/ihme/covid-19/deaths/dev/2020_05_16_mrtool/model_results.csv')
df = df.loc[df['location_id'].isin(us_locs)].reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])
df.loc[df['location_id'].isin(wa_locs), 'location_id'] = 570
df = df.groupby(['location_id', 'date'], as_index=False)[[f'draw_{d}' for d in range(1000)]].sum()
df['date'] = df['date'].apply(lambda x: x - pd.Timedelta(days=8))
df['mean_deaths'] = np.nan
df['mean_deaths'][1:] = np.diff(df[[f'draw_{d}' for d in range(1000)]], axis=0).mean(axis=1)
df = df.loc[df['date'] != df.groupby('location_id')['date'].transform(min)]

a26_idx = df['date'] == pd.to_datetime('2020-04-26')
m07_idx = df['date'] == pd.to_datetime('2020-05-07')
last_idx = df['date'] == df.groupby('location_id')['date'].transform(max)

df = df.loc[a26_idx | m07_idx | last_idx, ['location_id', 'date', 'mean_deaths']]
df['last_day_of_data'] = df.groupby('location_id')['date'].transform(max)
df['date'] = df['date'].apply(lambda x: 'april26' if x == pd.to_datetime('2020-04-26') 
                                       else 'may07' if x == pd.to_datetime('2020-05-07')
                                       else 'last'
                             )

df = pd.pivot_table(df, index=['location_id', 'last_day_of_data'], values=['mean_deaths'], columns='date').reset_index()
df.columns = df.columns.get_level_values(0)[:2].to_list() + df.columns.get_level_values(1)[2:].to_list()

df['pct_change_april26_may07'] = ((df['may07'] - df['april26']) / df['april26']) * 100
df['pct_change_may07_lastday'] = ((df['last'] - df['may07']) / df['may07']) * 100

df = loc_df[['location_id', 'location_name']].merge(
    df[['location_id', 'last_day_of_data', 'pct_change_april26_may07', 'pct_change_may07_lastday']]
)

df.to_csv('/ihme/covid-19/deaths/dev/2020_05_16_mrtool/pct_change_april26_may07_lastday.csv', index=False)


In [7]:
df

,location_id,location_name,last_day_of_data,pct_change_april26_may07,pct_change_may07_lastday
0,523,Alabama,2020-05-14,9.726419,-10.842645
1,524,Alaska,2020-05-14,-23.272804,-24.033598
2,525,Arizona,2020-05-14,2.072902,-8.691672
3,526,Arkansas,2020-05-14,-57.284722,37.463541
4,527,California,2020-05-14,8.727927,6.857752
5,528,Colorado,2020-05-14,8.219659,12.112155
6,529,Connecticut,2020-05-14,-33.196382,-23.385068
7,530,Delaware,2020-05-14,-13.348118,5.738916
8,531,District of Columbia,2020-05-14,-0.282082,-9.655967
9,532,Florida,2020-05-14,-25.139426,-26.747453
